# Programming Assignment #2 - Indexer and Query Processor

- **Deadline:** June 2nd, 2025 23:59 via Moodle

## Overview

The goal of this assignment is to implement the indexer and query processor modules of a web search engine. In addition to the source code of your implementation, your submission must include a characterization of the index built for a mid-sized corpus and the results retrieved for a set of queries.

## Implementation

You must use Python 3.13 for this assignment. Your code must run in a virtual environment using only the libraries included in the provided `requirements.txt` file. Execution errors due to missing libraries or incompatible library versions will result in a zero grade. To make sure you have the correct setup, you can test it using the following commands

```bash
python3 -m venv pa2
source pa2/bin/activate
pip3 install -r /path/to/requirements.txt
```

## Indexer

Your implementation must include an `indexer.py` file, which will be executed in the same virtual environment described above, as follows:

```bash
python3 indexer.py -m <MEMORY> -c <CORPUS> -i <INDEX>
```

with the following arguments:

- `-m <MEMORY>`: the memory available to the indexer in megabytes.
- `-c <CORPUS>`: the path to the corpus file to be indexed.
- `-i <INDEX>`: the path to the directory where indexes should be written.

At the end of the execution, your indexer.py implementation must print a JSON document to standard output [$^1$][Note_1] with the following statistics:

- `Index Size`: the index size in megabytes;
- `Elapsed Time`: the time elapsed (in seconds) to produce the index;
- `Number of Lists`: the number of inverted lists in the index;
- `Average List Size`: the average number of postings per inverted list.

The following example illustrates the required output format:

```json
{
  "Index Size": 2354,
  "Elapsed Time": 45235,
  "Number of Lists": 437,
  "Average List Size": 23.4
}
```

### Document Corpus

The corpus to be indexed comprises structured representations (with id, title, descriptive text, and keywords) for a total of 4,641,784 named entities present in Wikipedia. These structured representations are encoded as `JSON` documents in a single `JSONL` file available for download.[$^2$][Note_2] To speed up development, you are encouraged to use a smaller portion of the corpus to test your implementation before you try to index the complete version.

### Indexing Policies

For each document in the corpus (the `-c` argument above), your implementation must parse, tokenize, and index it. Your implementation must operate within the designated memory budget (the `-m` argument) during its entire execution. [$^3$][Note_3] [$^4$][Note_4] This emulates the most typical scenario where the target corpus far exceeds the amount of physical memory available to the indexer. At the end of the execution, a final representation of all produced index structures (inverted index, document index, term lexicon) must be stored as three separate files, one for each structure, at the designated directory (the `-i` argument).

In addition to this workflow, your implementation **must abide by the following policies**, which will determine your final grade in this assignment:

1. _Pre-processing Policy_: To reduce the index size, your implementation **must perform stopword removal and stemming**. Additional preprocessing techniques can be implemented at your discretion.

2. _Memory Management Policy_: To ensure robustness, your implementation **must execute under limited memory availability**. To this end, it must be able to produce partial indexes in memory (respecting the imposed memory budget) and merge them on disk. [$^5$][Note_5]

3. _Parallelization Policy_: To ensure maximum efficiency, you **must parallelize the indexing process across multiple threads**. You may experiment to find an optimal number of threads to minimize indexing time rate while minimizing the incurred parallelization overhead.

4. _Compression Policy (extra)_: Optionally, **you may choose to implement a compression scheme for index entries** (e.g. gamma for docids, unary for term frequency) for maximum storage efficiency.

[Note_1]: https://en.wikipedia.org/wiki/Standard_streams#Standard_output_(stdout)
[Note_2]: https://www.kaggle.com/datasets/rodrygo/entities

[Note_3]: <> "The memory limit will be strictly enforced during grading. If your program exceeds it, it may be automatically terminated with an out-of-memory (OOM) error. To prevent this, use `psutil.Process(os.getpid()).memory_info().rss` to monitor your current memory usage (in bytes), and offload partial indexes to disk before allocating more memory as needed."

[Note_4]: <> "Note that the memory budget refers to the total memory available to your implementation, not only to the memory needed to store the actual index structures. As a reference lower bound, assume your implementation will be tested with `-m 1024`."

[Note_5]: https://en.wikipedia.org/wiki/External_sorting#External_merge_sort

## Query Processor

Your implementation must include a `processor.py` file, which will be executed in the previously described environment, as follows:

```bash
python3 processor.py -i <INDEX> -q <QUERIES> -r <RANKER>
```

with the following arguments:

- `-i <INDEX>`: the path to an index file.
- `-q <QUERIES>`: the path to a file with the list of queries to process.
- `-r <RANKER>`: a string informing the ranking function (either “TFIDF” or “BM25”) to be used to score documents for each query.

After processing **each query** (the `-q` argument above), your `processor.py`
implementation must print a `JSON` document to standard output [$^6$][Note_6] with the top
results retrieved for that query according to the following format:

- `Query`, the query text;
- `Results`, a list of results.

Each result in the Results list must be represented with the fields:

- `ID`, the respective result ID;
- `Score`, the final document score.

The following example illustrates the required output format for a query:

```json
{
  "Query": "information retrieval",
  "Results": [
    { "ID": "0512698", "Score": 24.2},
    { "ID": "0249777", "Score": 12.4},
    ...
  ]
}
```

[Note_6]: https://en.wikipedia.org/wiki/Standard_streams#Standard_output_(stdout)

The results list for a query must be sorted in reverse document score order and include up to the top 10 results retrieved for that query.

### Query Processing Policies

For each query in the list provided via the `-q` argument, your implementation must pre-process the query, retrieve candidate documents from the given index (the `-i` argument), score these documents according to the chosen ranking model (the `-r` argument), and print the top 10 results using the aforementioned format. In addition to this standard workflow, **your implementation must abide by the following policies**:

1. _Pre-processing Policy_: Your implementation **must pre-process queries with stopword removal and stemming**. This policy should be aligned with the implemented document pre-processing policy for indexing to correctly match queries with documents.
2. _Matching Policy_: For improved efficiency, your implementation **must perform a conjunctive document-at-a-time (DAAT) matching** when retrieving candidate documents.
3. _Scoring Policy_: Your implementation **must provide two scoring functions: TFIDF and BM25**. You are free to experiment with different variants of these functions from the literature.
4. _Parallelization Policy (extra)_: To ensure maximum efficiency, you **may parallelize the query processing across multiple threads**. You may experiment to find an optimal number of threads to maximize your throughput while minimizing the incurred parallelization overhead.

## Deliverables

Before the deadline (June 2nd, 2025 23:59), you must submit a package file (`zip`) via Moodle containing the following:

1. Source code of your implementation;
2. Documentation file (`pdf`, max 3 pages);
3. Link to the produced index structures (stored on Google Drive).

Your `indexer.py` and `processor.py` files must be located at the root of your submitted zip file. You must guarantee that the index generated by your `indexer.py` can be correctly processed by your `processor.py`.

## Grading

This assignment is worth a total of 15 points distributed as:

- 10 points for your _implementation_, assessed based on the quality of your source code, including its overall organization (modularity, readability, indentation, use of comments) and appropriate use of data structures, as well as on how well it abides by the aforementioned indexing and query processing policies.
- 5 points for your documentation, assessed based on a short (pdf) report [$^7$][Note_7] describing your implemented data structures and algorithms, their computational complexity, as well as a discussion of their empirical efficiency (e.g. the time elapsed during each step of indexing and query processing, the speedup achieved via parallelization). Your documentation should also include a characterization of your produced inverted index, including (but not limited to) the following statistics: number of documents, number of tokens, number of inverted lists, and a distribution of the number of postings per inverted list. Likewise, you should include a characterization of the results produced for the provided test queries, such as the number of matched documents per query as well as statistics of the score distributions for the two implemented ranking functions (TFIDF and BM25).

[Note_7]: https://portalparts.acm.org/hippo/latex_templates/acmart-primary.zip "Your documentation should be no longer than 3 pages and use the ACM LATEX template (sample-sigconf.tex)"

## Late Submissions

Late submissions will be penalized in $2^{(d-1)} - 0.5$ points, where $d > 0$ is the number of days late. In practice, a submission 5 or more days late will result in a zero grade.

## Teams

This assignment must be performed **individually**. Any sign of plagiarism will be investigated and reported to the appropriate authorities.

---


# Resumindo

- Fazer Indexer
- Fazer Query Processor
- Caracterização do index
- Usar venv

## Indexer

- `indexer.py`
- 3 argumentos
  - `-m <MEMORY>`: a memória disponível para o indexer em megabytes.
  - `-c <CORPUS>`: o caminho para o arquivo do corpus a ser indexado.
  - `-i <INDEX>`: o caminho para o diretório onde os índices devem ser escritos.
- generate JSON
  - `Index Size`: o tamanho do índice em megabytes;
  - `Elapsed Time`: o tempo decorrido (em segundos) para produzir o índice;
  - `Number of Lists`: o número de listas invertidas no índice;
  - `Average List Size`: o número médio de postagens por lista invertida.

### Corpus

- id, title, descriptive text, and keywords
- 4,641,784 wikipedia entities
- Usar um mini-corpus para testar

### Indexing Policies

- Para cada documento:
  - parse
  - tokenize
  - index
- Respeitar o limite de memória
- Produzir 3 arquivos separados
  - Índice invertido
  - Index de documentos
  - Lexicon de termos
- **Policies:**
  1. Pre-processamento
    - Stopword removal
    - Stemming
    - Outras técnicas
  2. Gerenciamento de memória
    - Produzir índices parciais em memória
    - Fazer merge no disco
    - `psutil.Process(os.getpid()).memory_info().rss`
    - Assuma `-m 1024`
  3. Paralelização
    - Indexação em múltiplas threads
  4. Compressão (extra)
     - Implementar compressão para entradas do índice (gamma, unary, etc.)

## Query Processor

- Arquivo `processor.py`
- 3 argumentos
  - `-i <INDEX>`: o caminho para um arquivo de índice.
  - `-q <QUERIES>`: o caminho para um arquivo com a lista de consultas a serem processadas.
  - `-r <RANKER>`: uma string informando a função de ranqueamento (TFIDF ou BM25).
- produzir `JSON` para cada consulta: `{'query': 'IR', 'results': [{'id': '0', 'score': 1.0}, ...]}`
- Mostrar os 10 melhores resultados (ordenados do maior pro menor score)
- **Policies:**
  1. Pre-processamento
     - Stopword removal
     - Stemming
  2. Matching
     - DAAT conjuntivo
  3. Scoring
     - TFIDF
     - BM25
     - Outras variantes da literatura
  4. Parallelization (extra)
     - Query processing em múltiplas threads

## Deliverables

- Código Fonte (10 pontos)
  - Link pros arquivos gerados pelo indexer (Google Drive)
    - Índice invertido
    - Index de documentos
    - Lexicon de termos
  - na raiz:
  - `indexer.py`
  - `processor.py`
- Documentação (pdf, 3 páginas) (5 pontos)

---


# Código


## Indexer


In [ ]:
""" Generate a partial corpus from a given text file. """

import os

def generate_partial_corpus(input_file, output_file, num_lines):
    """Generate a partial corpus from the input file."""
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    with open(output_file, 'w', encoding='utf-8') as outfile:
        outfile.writelines(lines[:num_lines])

    print(f"Partial corpus generated with {num_lines} lines in {output_file}.")


generate_partial_corpus('source/input/corpus.jsonl', 'source/input/10k_corpus.jsonl', 10000)

In [ ]:
""" asd """
import json

def analyse_corpus():
    """Analyse the corpus and return a dictionary with the results."""
    path = "source/input/corpus.jsonl"

    with open(path, "r", encoding="utf-8") as file:
        lines = file.readlines()
        print(len(lines))
        for i, line in enumerate(lines):
            if i < 5:
                msg = line.strip() + "\n"
                msg += str(json.loads(line)) + "\n"
                print(msg)
                # print(line.keys())
    
analyse_corpus()

# Gerenciamento de Memória

In [ ]:
def doc_processing(doc, mem_limit, index_path):
    """ Process a single document """
    # Limpa variáveis de execuções anteriores
    local_vars = ['doc_info', 'parsed_doc', 'tokens', 'index_entries']
    for var in local_vars:
        if var in locals():
            del locals()[var]

    # Processa o documento
    doc = json.loads(doc)
    # ...resto do código...

In [ ]:
import gc


def doc_processing(doc, mem_limit, index_path):
    """ Process a single document """
    # Força coleta de lixo
    gc.collect()

    doc = json.loads(doc)
    # ...resto do código...

In [ ]:
import gc


def doc_processing(doc, mem_limit, index_path):
    """ Process a single document """
    # Limpa variáveis explicitamente
    local_vars = ['doc_info', 'parsed_doc', 'tokens', 'index_entries']
    for var in local_vars:
        if var in locals():
            del locals()[var]

    # Força coleta de lixo
    gc.collect()

    doc = json.loads(doc)
    # ...resto do código...

In [ ]:
import psutil
process = psutil.Process()
memory_info = process.memory_info().rss

# Bibliotecas

## NLTK

- works with human language data
- suite of text processing libraries for:
  - classification
  - [tokenization](https://www.nltk.org/api/nltk.tokenize.html)
    - `nltk.word_tokenize(sentence)`
  - [stemming](https://www.nltk.org/api/nltk.stem.html#module-nltk.stem)
  - tagging
  - parsing
  - semantic reasoning
  - Other
    - Vocabulary
    - [JSON](https://www.nltk.org/api/nltk.jsontags.html)
    - Stopwords
      - `import nltk`
      - `from nltk.corpus import stopwords`
      - `nltk.download('stopwords')`
      - `print(stopwords.words('english'))`

In [1]:
import nltk

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
# import nltk
from nltk.corpus import stopwords
import json

# nltk.download('stopwords')
nltk_stopwords = stopwords.words('english')

with open('source/input/stopwords.json', 'r', encoding='utf-8') as f:
    my_stopwords = json.load(f)

print("nltk stopwords:", len(nltk_stopwords))
print("my stopwords:", len(my_stopwords))

print(sorted(nltk_stopwords))
print(sorted(my_stopwords))

nltk stopwords: 198
my stopwords: 179
['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", "he's", 'her', 'here', 'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', "i'd", "i'll", "i'm", "i've", 'if', 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're

In [7]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\joaov\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True